# A Probabilistic Model for YeastGates

### Problem Challenge
We have 5 strains in the YeastGates dataset: WT, XOR_00, XOR_01, XOR_10, XOR_11. Each strain has corresponding "observed" measurements, namely the flow cytometry data for a population-level study, RNAseq data for transcriptomics, and LC-MS data for proteomics and metabolomics. These omics capture data at **different granularities** (some are more high-throughput than others).
Moreover, there is **inherent variation** in the datasets at multiple levels.
1. within-experiment variations (biological and technical replicates)
2. within-TA3 variations (robotic versus manual)
3. inter-TA3 variations (slightly different protocols: data in log phase or after stationary phase?)

*Research Question:* How do we principally account for these variations, and aggregate these **multiple omics** spaces to put out a predictive model of the YeastGates circuit?

### Proposed Solution
We use a probabilistic approach that:
1. Identifies key variables that show noise
2. Hypothesizes a latent space where the YeastGates strains "live" in a mixture model
3. Generates every observed output space from the latent space through a probabilistic map
4. Since we can marginalize over the latent space, we can work with data of any granularity

To define this model, we need to define the random variables and the distributions they follow. We define the latent space to have a simple K-component Gaussian mixture prior distribution. We want the latent space to be as interpretable and constrained as possible (we shift most degrees of freedom to the probabilistic mappings), so we let it have 2 isotropic dimensions. But in general, every data-point $x_n \in \mathbb{R}^d$ in this d-dimensional latent space follows the following distribution:

\begin{equation*}
x_n | \pi, \mu, \sigma \sim \sum_{k=1}^K \pi_k Normal(x_n | \mu_k, \sigma_k^2 \mathbf{1})
\end{equation*}

To define the probabilistic map(s) from the input latent space to the output observed space, we tried multiple options. Namely a Guassian process mapping, which treats the map as a random variable from a Gaussian Process prior following a certain mean function and a covariance function defined by a kernel (like RBF) applied to the input space. However on toy datasets, we observed it was much more difficult to optimize. (Largely because being a non-parametric map, it doesn't scale to large datasets.) Next, we tried a simpler linear map, akin to probabilistic PCA. This is a parametric map, whose weights depend only on the dimensions of input/output spaces. (Interestingly, it is a special case of a Gaussian process map when using a linear covariance kernel.)

So if we have a point in the latent space $x_n \in \mathbb{R}^D$, the corresponding point in the $j$th output space $y_n^j \in \mathbb{R}^P$ (with weight matrix $W^j \in \mathbb{R}^{P \times D}$) is given by the distribution:

\begin{equation*}
y_n^j | W^j,x_n,\sigma \sim Normal(y_n | W^jx_n^T, \sigma^2 \mathbf{1})
\end{equation*}

(Note that we've skipped the prior distributions on each of the parameters of this model, for brevity. But the weights, mixture distribution $\pi$, component distributions $\mu$, $\sigma$ have appropriate conjugate priors on them.)

Once we define these maps for all output spaces, we can ask some interesting arbitrary probabilistic queries to this model. For example, 
1. Given a FACS readout, what is the likely YeastGates strain?
2. Given a FACS readout, what is the likely metabolomic state?
3. Given a FACS readout in one protocol space, what is it likely gonna look like in the other one?
4. ...

### Building the Model

We use [Edward](http://edwardlib.org/), a deep probabilistic programming language built as a wrapper around [TensorFlow](https://www.tensorflow.org/), for writing our model and performing inference. This is a walkthrough to performing this analysis, which still needs some optimizing over! (Also, optimizing probabilistic models is kind of hard...)

First, to install some key Python packages.

In [ ]:
%%bash
pip install --user tensorflow
pip install --user edward
pip install --user plotly

For now, we use the FACS data from one of the TA3 performers (UW BioFab) to demonstrate the model. Alternatively, you can use the toy dataset function to play with a small dataset. 

In [ ]:
import pandas as pd #to import data as a dataframe
import numpy as np #for numerical computations

In [ ]:
#datapath
facs_bioworks = ['/work/05263/sloomba/jupyter/sd2e-community/shared-q0-hackathon/yeast-gates/flowDataFrames/UWBiofab/UWBiofab_20170811_flowDataFrame.csv', \
                 '/work/05263/sloomba/jupyter/sd2e-community/shared-q0-hackathon/yeast-gates/flowDataFrames/UWBiofab/UWBiofab_20170929_flowDataFrame.csv']

dfRun1 = pd.read_csv(facs_bioworks[0], index_col=0)
dfRun2 = pd.read_csv(facs_bioworks[1], index_col=0)

strains = ['WT', 'XOR_00', 'XOR_01', 'XOR_10', 'XOR_11'] #YeastGates families
channels = ['FL1-A','FSC-A','SSC-A'] #we are only using the GFP fluorescence channel, and the forward and side scatters
facs = []
labels = []
for i in range(len(strains)):
    facs.append(dfRun1[dfRun1['Strain'] == strains[i]][channels])
    labels.append(np.ones(np.shape(facs[-1])[0])*i)
    facs.append(dfRun2[dfRun2['Strain'] == strains[i]][channels])
    labels.append(np.ones(np.shape(facs[-1])[0])*i)
facs = [np.concatenate(facs)] #we use only one output space as a proof of principle, but in general this will be a list of multiple output spaces
labels = [np.concatenate(labels)]

We define the fixed hyperparameters of the model here.

In [ ]:
NUM_FAMILIES = len(strains)
NUM_LATENT_DIMS = 2
SIZE_OBS_SPACES = [np.shape(fac)[1] for fac in facs] # <FACS1, FACS2, ...>
NUM_OBS_SPACES = len(SIZE_OBS_SPACES)
N_OBS_SPACES = [np.shape(fac)[0] for fac in facs] # number of data points in each observed space
MINI_BATCH_SIZE = 100
NUM_MCMC_SAMPLES = 10000
y_train = facs
z_train = labels

MU_SCALE = 8.0 #variance of the prior on component means; the higher the more separate the clusters might be
WT_SCALE = 4.0 #variance of the prior on map weights; the higher, the more the degree of freedom in weight space

In [ ]:
#toy dataset
def build_toy_dataset(fixed=True):
    if fixed:
        pi = [np.array([0.4, 0.6]), np.array([0.3, 0.7])]
        mus = [[[1, 1], [-1, -1]], [[1.5, 1.25], [-0.75, -1.25]]]
        stds = [[[0.1, 0.2], [0.2, 0.1]], [[0.2, 0.1], [0.1, 0.2]]]
    else:
        pi = [np.random.beta(1, 1, NUM_FAMILIES) for i in range(NUM_OBS_SPACES)]
        pi = [pi_c/np.sum(pi_c) for pi_c in pi]
        mus = [np.random.randn(NUM_FAMILIES, SIZE_OBS_SPACES[i]) for i in range(NUM_OBS_SPACES)]
        stds = [np.random.gamma(1, 1, (NUM_FAMILIES, SIZE_OBS_SPACES[i])) for i in range(NUM_OBS_SPACES)]
    y_trains = [np.zeros((N_OBS_SPACES[i], SIZE_OBS_SPACES[i]), dtype=np.float32) for i in range(NUM_OBS_SPACES)]
    z_trains = [np.zeros(N_OBS_SPACES[i], dtype=np.int32) for i in range(NUM_OBS_SPACES)]
    for i in range(NUM_OBS_SPACES):
        for j in range(N_OBS_SPACES[i]):
            k = np.argmax(np.random.multinomial(1, pi[i]))
            y_trains[i][j, :] = np.random.multivariate_normal(mus[i][k], np.diag(stds[i][k]))
            z_trains[i][j] = k
    return (y_trains, z_trains)

#y_train, z_train = build_toy_dataset()

We plot the observed FACS data in a 3D space. Note that the data is quite large (~75K data-points), and so we plot only a random subset.

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [ ]:
index = np.random.choice(np.shape(y_train[0])[0], 500)
traces = [go.Scatter3d(x = y_train[i][index,0], y = y_train[i][index,1], z = y_train[i][index,2], \
                     showlegend=False, mode = 'markers', marker=dict(size=10, color=z_train[i][index])) \
          for i in range(NUM_OBS_SPACES)]
init_notebook_mode()
iplot(traces, filename='training_data')

We can now begin building our model. We import TensorFlow and Edward libraries, and the model distributions which define our random variables.

In [ ]:
import tensorflow as tf

hello = tf.constant('hello world') #to check if TensorFlow was imported correctly
sess = tf.Session()
print(sess.run(hello))

import edward as ed
from edward.models import Dirichlet, InverseGamma, MultivariateNormalDiag, Normal, ParamMixture, MultivariateNormalTriL, Gamma, \
PointMass, Empirical
from edward.util import rbf # for the Gaussian Process kernel, not used at the moment

We now define the random variables of the latent space, following certain prior distributions.

In [ ]:
#Mixture Model on the Latent Space
alpha = 1.0
pi = Dirichlet(tf.constant([alpha]*NUM_FAMILIES))
mu = Normal(loc=tf.zeros(NUM_LATENT_DIMS), scale=MU_SCALE*tf.ones(NUM_LATENT_DIMS), sample_shape=NUM_FAMILIES)
sigma = InverseGamma(concentration=tf.ones(NUM_LATENT_DIMS), rate=tf.ones(NUM_LATENT_DIMS), sample_shape=NUM_FAMILIES)
x_latent = [ParamMixture(mixing_weights=pi, component_params={'loc': mu, 'scale_diag': tf.sqrt(sigma)}, \
                        component_dist=MultivariateNormalDiag, sample_shape=MINI_BATCH_SIZE)]*NUM_OBS_SPACES
z_assignments = [x.cat for x in x_latent] #this variable represents the assignment of a data-point to a certain YeastGates strain

#x_latent = [Normal(loc=tf.zeros(NUM_LATENT_DIMS), scale=tf.ones(NUM_LATENT_DIMS), sample_shape=MINI_BATCH_SIZE)]*NUM_OBS_SPACES
print(x_latent)
print(z_assignments)

We now define the random variables of the probabilistic map to output space(s), following certain prior distributions.

(Note above that we have defined the sample shape for the latent space of a `MINI_BATCH_SIZE` size, and not the actual number of outputs. This is because we would be performing batch learning, as further explained below.) 

In [ ]:
#Probabilistic PCA map to latent space
weights = [Normal(loc=tf.zeros([SIZE_OBS_SPACES[i], NUM_LATENT_DIMS]), scale=WT_SCALE*tf.ones([SIZE_OBS_SPACES[i], NUM_LATENT_DIMS])) \
           for i in range(NUM_OBS_SPACES)]
y_obs = [Normal(loc=tf.matmul(weights[i], x_latent[i], transpose_b=True), scale=tf.Variable(tf.ones([SIZE_OBS_SPACES[i], MINI_BATCH_SIZE]))) \
         for i in range(NUM_OBS_SPACES)]
                  
print(weights)
print(y_obs)

Our model definition is now in place. The "learning" in probabilistic models is basically the inference of the posterior distributions, given the observed data. There are different inference algorithm strategies that can be used, all of which involve defining a "proxy" distribution which captures the same aggregate statistics as the actual distribution. We'd be doing Gibbs sampling for the parameters of the mixture model, which is a kind of MCMC sampling strategy. We define empirical distributions for the unknown random variables below.

In [ ]:
#Inference of assignments in latent space
pi_q = Empirical(params=tf.Variable(tf.ones([NUM_MCMC_SAMPLES, NUM_FAMILIES])/NUM_FAMILIES))
mu_q = Empirical(params=tf.Variable(tf.zeros([NUM_MCMC_SAMPLES, NUM_FAMILIES, NUM_LATENT_DIMS])))
sigma_q = Empirical(params=tf.Variable(tf.ones([NUM_MCMC_SAMPLES, NUM_FAMILIES, NUM_LATENT_DIMS])))

print(pi)
print(pi_q)
print(mu)
print(mu_q)
print(sigma)
print(sigma_q)

(Note that the actual and the proxy distributions should have the same shape!) Now we define similar proxy distributions for the parameters of the probabilistic map, namely the latent space points and the weights. We will be using a variational inference strategy here, which tries to maximize a "simpler" likelihood function than the actual one. We use the Normal distributions to define these simpler likelihoods. 

**Mini Batching:** Since some of our output spaces have a lot of data, like the FACS, inference can be very slow if we try to maximize the likelihood over all data-points at once. So, we do a mini-batching of our space. Which means we consider only a random subsample of the observations in one iteration of the inference algorithm, and choose a new one on the next one. Since our data in consideration keeps changing, we need to define "placeholders" for them.

In [ ]:
#placeholders for mini-batching the code
y_ph = [tf.placeholder(tf.float32, [SIZE_OBS_SPACES[i], MINI_BATCH_SIZE]) for i in range(NUM_OBS_SPACES)]
z_ph = [tf.placeholder(tf.int32, MINI_BATCH_SIZE) for i in range(NUM_OBS_SPACES)]
idx_ph = [tf.placeholder(tf.int32, MINI_BATCH_SIZE) for i in range(NUM_OBS_SPACES)]

#approximate distributions for inference
weights_variables = [[tf.Variable(tf.random_normal([SIZE_OBS_SPACES[i], NUM_LATENT_DIMS])), \
                     tf.Variable(tf.random_normal([SIZE_OBS_SPACES[i], NUM_LATENT_DIMS]))] for i in range(NUM_OBS_SPACES)]
weights_q = [Normal(loc=weights_variables[i][0], scale=tf.nn.softplus(weights_variables[i][1])) for i in range(NUM_OBS_SPACES)]
x_latent_variables = [[tf.Variable(tf.random_normal([N_OBS_SPACES[i], NUM_LATENT_DIMS])), \
                       tf.Variable(tf.random_normal([N_OBS_SPACES[i], NUM_LATENT_DIMS]))] for i in range(NUM_OBS_SPACES)]
x_latent_q = [Normal(loc=tf.gather(x_latent_variables[i][0], idx_ph[i]), \
                     scale=tf.nn.softplus(tf.gather(x_latent_variables[i][1], idx_ph[i]))) for i in range(NUM_OBS_SPACES)]

Once we've defined the proxy distributions, we are ready to tie them to the actual model distributions. We do this in Edward using dictionaries. We define the data dictionary, which ties the distributions of observed data with the placeholders of data. We define the variable dictionary, which ties the actual and proxy distributions of rest of the model parameters.

Note that we would be using an EM-style strategy to perform inference, wherein we first infer the probabilistic map while keeping the mixture model constant, and then vice-versa, reiteratively. And so we define two sets of data and variable dictionaries.

In [ ]:
#data and variable dicts for inferring mixture model
data_dict1 = dict(zip(y_obs, y_ph))
data_dict1.update(zip(z_assignments, z_ph))
data_dict1.update({pi: pi_q, mu: mu_q, sigma: sigma_q})
variable_dict1 = dict(zip(x_latent, x_latent_q))
variable_dict1.update(zip(weights, weights_q))

#data and variable dicts for inferring probabilistic map
data_dict2 = dict(zip(y_obs, y_ph))
data_dict2.update(zip(z_assignments, z_ph))
data_dict2.update(zip(x_latent, x_latent_q))
data_dict2.update(zip(weights, weights_q))
variable_dict2 = {pi: pi_q, mu: mu_q, sigma: sigma_q}

#defining the 2 inference engines
inference_p = ed.Gibbs(variable_dict2, data=data_dict2)
inference_x = ed.KLqp(variable_dict1, data=data_dict1)

#defining a scaling dictionary, necessary for scaling updates in batch algorithms
scaling_dict = dict(zip(x_latent, [float(sum(N_OBS_SPACES))/(MINI_BATCH_SIZE*NUM_OBS_SPACES)]*NUM_OBS_SPACES))
scaling_dict.update(zip(y_obs, [float(sum(N_OBS_SPACES))/(MINI_BATCH_SIZE*NUM_OBS_SPACES)]*NUM_OBS_SPACES))

#initializing the inference engines
inference_p.initialize(scale=scaling_dict)
inference_x.initialize(scale=scaling_dict, n_samples = 5)

We are now ready to begin inference. We define an Edward session to run the inference in, and iteratively do variational inference and Gibbs sampling to simultaneously learn the mixure model and probabilistic map(s).

(Note, if the codeblock below throws a placeholder error, just rerun it.)

In [ ]:
sess = ed.get_session() #init Edward sessions
tf.global_variables_initializer().run() #to initialize all variables in the computational graph of TensorFlow
for _ in range(inference_p.n_iter): #main inference loop
    idx_batch = [np.random.choice(N_OBS_SPACES[i], MINI_BATCH_SIZE) for i in range(NUM_OBS_SPACES)] #find a random mini-batch
    y_obs_batch = [np.transpose(y_train[i][idx_batch[i],:]) for i in range(NUM_OBS_SPACES)] #mini batch of observed data
    z_ass_batch = [z_train[i][idx_batch[i]] for i in range(NUM_OBS_SPACES)] #mini batch of observed assignments
    feeding_dict = dict(zip(y_ph, y_obs_batch)) #feeding the batched data to appropriate placeholder variables in a feeding dictionary
    feeding_dict.update(zip(z_ph, z_ass_batch))
    feeding_dict.update(zip(idx_ph, idx_batch))
    for _ in range(5): #variational inference update 
        info_dict = inference_x.update(feed_dict=feeding_dict)
    info_dict = inference_p.update(feed_dict=feeding_dict) #Gibbs sampling update
    t = info_dict['t']
    if t % 100 == 0:
        print("\nweights:")
        [print(weight[0].eval()) for weight in weights_variables]
        print("pi", tf.reduce_mean(pi_q.params, 0).eval())
        print("mu", tf.reduce_mean(mu_q.params, 0).eval())
        print("sigma", tf.reduce_mean(sigma_q.params, 0).eval())

We can vary the number of iterations for our algorithm by changing the `NUM_MCMC_SAMPLES` variable defined above. We can now plot a small subset of the inferred latent space.

In [ ]:
x_test = np.concatenate([x_latent_variables[i][0].eval() for i in range(NUM_OBS_SPACES)]) #evaluating the latent space
w_test = np.concatenate([weights_variables[i][0].eval() for i in range(NUM_OBS_SPACES)]) #evaluating the weights of probabilistic map
index = np.random.choice(np.shape(x_test)[0], MINI_BATCH_SIZE)
x_test = x_test[index]
traces = [go.Scatter(x = x_test[:,0], y = x_test[:,1], showlegend=False, mode = 'markers', \
                     marker=dict(size=10, color=z_train[0][index]))]
init_notebook_mode()
iplot(traces, filename='latent_space')
print(np.shape(x_test))
print(w_test)

We are also interested in finding out how well the YeastGates strains differentiate out in the latent space. This can be done by evaluating the mixture components. These components can also be ways of diagnosing how much the different observed spaces influence this latent space, alone or in combinations, thus providing a way to compare different heterogenous observed spaces in this latent model space.

In [ ]:
print("pi", tf.reduce_mean(pi_q.params, 0).eval())
print("mu", tf.reduce_mean(mu_q.params, 0).eval())
print("sigma", tf.reduce_mean(sigma_q.params, 0).eval())

We can also visualize the latent space by plotting the component variances in the latent space.

In [ ]:
def plot_components(means, covariances):
    import math
    color_iter = ['navy', 'cyan', 'cornflowerblue', 'gold', 'orange']
    data = []
    for (mean, covar, color) in (zip(means, covariances, color_iter)):
        a = covar[0]
        b = covar[1]
        x_origin = mean[0]
        y_origin = mean[1]
        x_ = [ ]
        y_ = [ ]    
        for t in range(0,361,10):
            x = a*(math.cos(math.radians(t))) + x_origin
            x_.append(x)
            y = b*(math.sin(math.radians(t))) + y_origin
            y_.append(y)
        data.append(go.Scatter(x=x_ , y=y_, mode='lines', showlegend=False, line=dict(color=color, width=2)))
    return data

init_notebook_mode()
iplot(plot_components(tf.reduce_mean(mu_q.params, 0).eval(), tf.reduce_mean(sigma_q.params, 0).eval()), filename='latent_space')

We are still far from coming up with solid hypotheses on actual YeastGates data! But we tried to provide a principled probabilistic framework to aggregate arbitrary observed data collected from different sources.